In [2]:
import price_data as price
import chart
import pandas as pd
import numpy as np
import ccxt
from datetime import datetime
from users import users
from ftx_client import FtxClient
import math

ftx_ccxt = ccxt.ftx({
    'apiKey': 'mFRyLR4AAhLTc5RlWov3PKTcIbMHw3vGZwiHnsrn',
    'secret': 'oKaY1WEqTuhnNnq0iRi_Ry-CYckvE89-gPUPf21B',
    'enableRateLimit': True
})
ftx_ccxt.password = 'Ev443978??'
cex= ccxt.cex({
    'uid' : 'up109520414',
    'apiKey': '1X2uEcPlvBCe4CcMtzWKguG1SDI',
    'secret': '8JY4fDg6hRz0DTolZHz77XPdC1o',
    'enableRateLimit': True,
    })

ftx_ccxt.headers = {'FTX-SUBACCOUNT':'Savings'}

def get_limit(symbol, markets):
    return float(next(filter(lambda x:x['symbol']==symbol, markets))['limits']['amount']['min'])


In [3]:
def accumulate(risk, risk_ethbtc, exchange_str, exchange, symbols):
    usd_balance=exchange.fetch_partial_balance('USD')['free']
    gbp_balance=exchange.fetch_partial_balance('GBP')['free']
    markets=exchange.fetch_markets()
    print('USD balance: ', usd_balance)
    if risk>=0.5:
        print('ETHBTC risk level', risk_ethbtc)
        #sell btc
        price=price_data.get_price_data('1m', symbol='BTC/USD')
        try:
            balance=float(exchange.fetch_partial_balance('BTC')['total'])
        except:
            print('No balance for BTC ')
        current_price=price.iloc[-1]['close'].item()
        dynamic_sell_amount=round((-1/(50*(risk-1))), 2)*balance
        exchange.create_limit_sell_order('BTC/USD',dynamic_sell_amount,current_price)
        print('Sold %s of %s' %(dynamic_sell_amount, 'BTC/USD'))
        usd_amount=dynamic_sell_amount*current_price
        if risk_ethbtc<0.5: #move those profits over to eth
                usd_balance=exchange.fetch_partial_balance('USD')['free']   
                eth_price=price_data.get_price_data('1m',symbol='ETH/USD')       
                #buy
                current_eth_price=eth_price.iloc[-1]['close'].item()
                buy_amount=usd_amount/current_eth_price
                if buy_amount<usd_balance:
                    print('Buying eth with BTC profits')
                    amount_to_buy=round(buy_amount/current_eth_price,3) #eth has a precision of 0.001
                    limit=get_limit('ETH/USD', markets)
                    if amount_to_buy>limit:
                        # exchange.create_limit_buy_order('ETH/USD',amount_to_buy,current_price)
                        print('Bought %s of %s' %(amount_to_buy, 'ETH/USD'))
                    elif limit<usd_balance:
                        print('Amount not above limit: %s, %s' % (limit,amount_to_buy))
                        # exchange.create_limit_buy_order('ETH/USD',limit,current_price)
                        print('Bought %s of %s' %(limit, 'ETH/USD'))
                    else:
                        print('Minimum amount was above usd balance.')

        else:#start selling alts
            #selling
            for i in range(len(symbols)-1): #sells everything except btc
                symbol=symbols[i+1]
                price=price_data.get_price_data('1m', symbol=symbol, exchange_str=exchange_str)
                noDecimals=np.absolute(np.log10(next(filter(lambda x:x['symbol']==symbol, exchange.fetch_markets()))['precision']['amount']))
                try:
                    balance=float(exchange.fetch_partial_balance(symbol.split('/')[0])['total'])
                except:
                    print('No balance for : ', symbol)
                current_price=price.iloc[-1]['close'].item()
                dynamic_sell_amount=round(round((-1/(50*(risk_ethbtc-1))), 2)*balance,noDecimals)
                # exchange.create_limit_sell_order(symbol,dynamic_sell_amount,current_price)
                print('Sold %s of %s' %(dynamic_sell_amount, symbol))



    elif risk<0.5 and usd_balance>daily_buy_amount:
        list_of_limits = [get_limit(symbol, markets) for symbol in symbols]
        max_min_amount = max(list_of_limits)
        for i in range(len(symbols)):
            usd_balance=exchange.fetch_partial_balance('USD')['free']
            symbol=symbols[i]
            print('Symbol:', symbol)
            price=price_data.get_price_data('1m', symbol=symbol, exchange_str=exchange_str)
            noDecimals=np.absolute(np.log10(next(filter(lambda x:x['symbol']==symbol, exchange.fetch_markets()))['precision']['amount']))
            current_price=price.iloc[-1]['close'].item()
            #buy
            dynamic_buy_amount=round(daily_buy_amount*((0.5/(risk-0.15))-1),2)
            if dynamic_buy_amount<usd_balance:
                amount_to_buy=round(dynamic_buy_amount/current_price,4)
                limit=get_limit(symbol, markets)
                if amount_to_buy>limit:
                    # exchange.create_limit_buy_order(symbol,amount_to_buy,current_price)
                    print('Bought %s of %s' %(amount_to_buy, symbol))
                elif limit<usd_balance:
                    print('Amount not above limit: %s, %s' % (limit,amount_to_buy))
                    # exchange.create_limit_buy_order(symbol,limit,current_price)
                    print('Bought %s of %s' %(limit, symbol))
                else:
                    print('Minimum amount was above usd balance.')
            else:
                print('No balance')
    else:
        print('Not doing anything')

In [4]:
for key, user in users.items():
    print(key)
    weekly=user['weekly']
    if weekly and datetime.today().weekday() == 0:
        buy_amount=user['daily_buy_amount']*7
    number_of_cryptos = 0
    for name, details in user['exchanges'].items():
        print(name)
        exchange: ccxt.Exchange
        if name=='ftx':
            exchange = ccxt.ftx(details['api'])
            if 'header' in details:
                exchange.headers = details['header']
        elif name=='cex':
            exchange = ccxt.cex(details['api'])
        symbols=details['symbols']
        number_of_cryptos = number_of_cryptos + len(symbols)
    print(buy_amount, number_of_cryptos, buy_amount/number_of_cryptos)
    m=5
    d = math.ceil((m*number_of_cryptos)/buy_amount)
    print(d)
        

rob
False
ftx
cex
70 5 14.0
1
edward
True
ftx
7 2 3.5
2


In [5]:
noDecimals=np.absolute(np.log10(next(filter(lambda x:x['symbol']=='MATIC/USD', ftx_ccxt.fetch_markets()))['precision']['amount']))

In [6]:
rob = ccxt.ftx(users['rob']['exchanges']['ftx']['api'])
if 'header' in users['rob']['exchanges']['ftx']:
    rob.headers = users['rob']['exchanges']['ftx']['header']
    print(rob.fetchPartialBalance('BTC'))

{'free': 0.01889786, 'used': 0.0, 'total': 0.01889786}


In [7]:
def transfer_ftx_to_cex(ftx_dict: dict,cex: dict,usd_amount: float,headers: dict=None, blockchain = 'SOL'):
    ftx = ccxt.ftx(ftx_dict)
    ftx.password=ftx_dict['password']
    cex = ccxt.cex(cex)
    if headers != None:
        ftx.headers = headers
    symbol=blockchain+'/'+'USD'
    current_price=price.get_current_price(symbol)
    amount = usd_amount/current_price
    noDecimals=int(np.absolute(np.log10(next(filter(lambda x:x['symbol']==symbol, ftx.fetch_markets()))['precision']['amount'])))
    amount_to_buy = round(amount,noDecimals)
    deposit_address = cex.fetchDepositAddress(blockchain)
    # ftx.createMarketBuyOrder(symbol='SOL/USD', amount=amount_to_buy)
    ftx.withdraw(code=blockchain, amount=amount_to_buy, address=deposit_address['address'])
    

my_ftx = {
    'apiKey': 'mFRyLR4AAhLTc5RlWov3PKTcIbMHw3vGZwiHnsrn',
    'secret': 'oKaY1WEqTuhnNnq0iRi_Ry-CYckvE89-gPUPf21B',
    'enableRateLimit': True,
    'password': 'Ev443978??'
}

my_cex = {
    'uid' : 'up109520414',
    'apiKey': '1X2uEcPlvBCe4CcMtzWKguG1SDI',
    'secret': '8JY4fDg6hRz0DTolZHz77XPdC1o',
    'enableRateLimit': True,
}

my_savings = {'FTX-SUBACCOUNT':'Savings'}
# transfer_ftx_to_cex(my_ftx,my_cex,50, my_savings)
edward = ccxt.ftx(users['edward']['exchanges']['ftx']['api'])
edward.fetch_partial_balance('USD')


{'free': 4.46752782, 'used': 0.0, 'total': 4.46752782}